shortest path respects direction

In [46]:

import networkx as nx


In [ ]:
# gives shortest dist for all nodes to all other nodes
def floyd_warshall(graph:nx.DiGraph):
    distances = {}
    nodes = list(graph.nodes)

    # initalise 2 key dict populated with inf distance
    for node1 in nodes:
        distances[node1]={}
        for node2 in nodes:
            distances[node1][node2] = float("inf")

    # set dist to nodes themselves to 0
    for node in nodes:
        distances[node][node]=0

    # set dist for every given edge
    # diffrence is for digraph it only populates in direction of one
    for edge in graph.edges.data():
            node1, node2, pdict = edge
            weight = pdict["weight"]
            distances[node1][node2]=weight
    
    # up to here distances is an adjacency matric as a dualkey dictionary
    # print(distances)


    # important for intermedate node loop to be on outside (?) might be absolutely nessicary for relaxation
    for intermediaryNode in nodes:
        for nodeI in nodes:
            for nodeJ in nodes:
                # sees if using intermediaryNode between nodeI to nodeJ is faster, if so add
                distances[nodeI][nodeJ] = min(distances[nodeI][nodeJ], distances[nodeI][intermediaryNode] + distances[intermediaryNode][nodeJ])
    
    # if still unresolved -> negative cycle
    for intermediaryNode in nodes:
        for nodeI in nodes:
            for nodeJ in nodes:
                if distances[nodeI][intermediaryNode] + distances[intermediaryNode][nodeJ] < distances[nodeI][nodeJ] :
                    return "negative cycle"
                # note this algo checks all start nodes, so will always return neg cycle if there is one
                # but for the single source ones, wont return if the neg cycle is never traversable from source node

    return distances


In [48]:
  # single node source  
# good for graphs with no negative edges
def dijkstra(graph: nx.DiGraph, start) -> dict:

    unvisited = list(graph.nodes)
    # gets list

    node_distances = {node: float('inf') for node in graph.nodes}
    # creates set of distances, default initiate to inf
    
    node_distances[start] = 0

    
    while len(unvisited) > 0:
    
        node = min(unvisited, key=lambda x: node_distances[x])
    
        for i in graph.successors(node):
    
            if i in unvisited:
    
                node_distances[i] = min(node_distances[i], node_distances[node] + graph[node][i]['weight'])
    
        unvisited.remove(node)


        # NEGATIVE CYCLE DETECTION FROM BELLAMFORD
        # detect negative cycle
        # basically, the graph will be fully relaxed/finalised by now as we relaexd |V|-1 times
        # so if there is still any futher possible shortest path, that means theres a negative cycle
        for edge in graph.edges.data():
                nodeU,nodeV,pdict=edge
                weight=pdict["weight"]

                if node_distances[nodeU] + weight < node_distances[nodeV]:
                    return "negative cycle"
    
    return node_distances




In [49]:
# single source
# good for graphs with negative edges (directed only) 
#  as all undir edges are cycles by themselves, undir cannot handle negative lengh
# will return if detects negative cycle 

def bellman_ford(graph: nx.DiGraph,start):
    if start not in graph.nodes:
        raise Exception("start node not in grpah")
    distances={}
    for node in graph.nodes:
        distances[node]=float('inf')
    distances[start]=0


    for _ in range(len(graph.nodes)-1):
        for edge in graph.edges.data():
            nodeU,nodeV,pdict=edge
            weight=pdict["weight"]
            if distances[nodeU] + weight < distances[nodeV]:
                distances[nodeV]=distances[nodeU] + weight


    # detect negative cycle
    # basically, the graph will be fully relaxed/finalised by now as we relaexd |V|-1 times
    # so if there is still any futher possible shortest path, that means theres a negative cycle
    for edge in graph.edges.data():
            nodeU,nodeV,pdict=edge
            weight=pdict["weight"]

            if distances[nodeU] + weight < distances[nodeV]:
                return "negative cycle"


    
    return distances


In [50]:
# testing

nodes=['a', 'b', 'c', 'd', 'e', 'f']
# edges=[('a', 'b', {'weight': 6}), ('a', 'd', {'weight': 5}), ('a', 'c', {'weight': 1}), ('b', 'c', {'weight': 5}), ('d', 'c', {'weight': 5}), ('b', 'e', {'weight': 3}), ('e', 'f', {'weight': 6}), ('e', 'c', {'weight': 6}), ('c', 'f', {'weight': 4}), ('d', 'f', {'weight': 2})]
# 
# with neg cyucle
edges=[('a', 'b', {'weight': -1}), ('a', 'd', {'weight': 5}), ('b', 'c', {'weight': -6}), ('c', 'a', {'weight': 5}), ('d', 'c', {'weight': 5}), ('b', 'e', {'weight': 3}), ('e', 'f', {'weight': 6}), ('e', 'c', {'weight': 6}), ('c', 'f', {'weight': 4}), ('d', 'f', {'weight': 2})]



ingraph=nx.DiGraph()


ingraph.add_nodes_from(nodes)

ingraph.add_edges_from(edges)

tests=nodes
for testvar in tests:
    print(f"test souuce node {testvar}")
    floydresult=floyd_warshall(ingraph)
    if type(floydresult) == dict:
        print(floydresult[testvar])
    else:
        print(floydresult)

    
    print(dijkstra(ingraph,testvar))
    print(bellman_ford(ingraph,testvar))
    print()

test souuce node a
-4
-2
negative cycle
negative cycle
negative cycle

test souuce node b
-4
-2
negative cycle
negative cycle
negative cycle

test souuce node c
-4
-2
negative cycle
negative cycle
negative cycle

test souuce node d
-4
-2
negative cycle
negative cycle
negative cycle

test souuce node e
-4
-2
negative cycle
negative cycle
negative cycle

test souuce node f
-4
-2
negative cycle
{'a': inf, 'b': inf, 'c': inf, 'd': inf, 'e': inf, 'f': 0}
{'a': inf, 'b': inf, 'c': inf, 'd': inf, 'e': inf, 'f': 0}

